In [1]:
import scipy.optimize
import pandas
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def opt(technique, specification, filename):
    
    def extract_data(filename):
        df = pandas.read_csv(filename)
        ydata = df.T1.values
        udata = df.Q1.values
        time = df.Time.values
        return ydata, udata, time
    
    def OF(x, filename):
        a1, a2, b1, b2 = x
        ydata, udata, time = extract_data(filename)
        
        ydev = ydata - ydata[0]
        udev = udata - udata[0]
        yt = []
        for i, it in enumerate(time):
            if i-1<0:
                y1 = ydev[0]
                u1 = udev[0]
            else:
                y1 = ydev[i-1]
                u1 = udev[i-1]
            if i-2<0:
                y2 = ydev[0]
                u2 = udev[0]
            else:
                y2 = ydev[i-2]
                u2 = udev[i-2]
            if i-3<0:
                y3 = ydev[0]
                u3 = udev[0]
            else:
                y3 = ydev[i-3]
                u3 = udev[i-3]
            yt.append(a1*y1 + a2*y2 + b1*u1 + b2*u2 )#+ a3*y3 + b3*u3)

        return sum((ydata - (yt+ydata[0]))**2)
    
    return scipy.optimize.technique(OF, specification)

        

In [ ]:
opt(differential_evolution, )